In [1]:
from source.custom_cnn.gh_custom_architecture import GH_CNN as CNN
from data.data_preperation import load_data
from training.parameters import get_loss_fun, get_optimizer
from training.process import train_one_epoch, validate
import torch
import numpy as np
from ray import tune
from ray.tune import CLIReporter,JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler, HyperBandScheduler 
from functools import partial


In [2]:
My_CNN = CNN()
train_loader,validation_loader = load_data()
#optimizer = get_optimizer(My_CNN)
loss_fn = get_loss_fun()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
EPOCHS =12
def train_with_parameters(config):
    My_CNN = CNN()
    train_loader,validation_loader = load_data(config['batch_size'])
    #optimizer = get_optimizer(My_CNN)
    loss_fn = get_loss_fun()

    best_vloss = 1_000_000.
    best_accuracy = 0
    #print(config['lr'], config =['momentum'])
    for epoch in range(EPOCHS):
        avg_loss = train_one_epoch(model=My_CNN,optimizer=get_optimizer(My_CNN,config['lr'],config['momentum']),loss_fn=loss_fn,train_loader=train_loader)
        avg_vloss,accuracy = validate(model= My_CNN,validation_loader=validation_loader,loss_fn=loss_fn) 
        print(epoch)  
        print("loss: train {}, test {}, accuracy on test {}".format(avg_loss,avg_vloss,accuracy))
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
        if best_accuracy < accuracy:
            best_accuracy=accuracy
    #tune.report(loss = best_vloss,mean_accuracy=best_accuracy)
    return best_accuracy,best_vloss

        
        

In [4]:
config = {"lr": 0.001, "momentum":0.7,'batch_size':8}
train_with_parameters(config)

Files already downloaded and verified
Files already downloaded and verified
0
loss: train 1.9262114182710648, test 1.8905808925628662, accuracy on test 32.34
1
loss: train 1.653115183055401, test 1.6310735940933228, accuracy on test 40.0
2
loss: train 1.522185061752796, test 1.5352693796157837, accuracy on test 44.3
3
loss: train 1.4328638684153556, test 1.3999102115631104, accuracy on test 49.29
4
loss: train 1.3667404010295867, test 1.342947244644165, accuracy on test 51.86
5
loss: train 1.2770145881474018, test 1.3167521953582764, accuracy on test 52.98
6
loss: train 1.2482667464613915, test 1.2864431142807007, accuracy on test 54.72
7
loss: train 1.209237706154585, test 1.2278218269348145, accuracy on test 56.33
8
loss: train 1.184493182271719, test 1.2070938348770142, accuracy on test 56.65
9
loss: train 1.1563770025968552, test 1.1867088079452515, accuracy on test 58.06
10
loss: train 1.1043328238129615, test 1.148362159729004, accuracy on test 59.96
11
loss: train 1.068216765940

(60.26, 1.119126319885254)

In [4]:
config = {"lr": 0.001, "momentum":0.7,'batch_size':8}
train_with_parameters(config)

Files already downloaded and verified
Files already downloaded and verified
0
loss: train 1.8882906171083451, test 1.838563323020935, accuracy on test 33.25
1
loss: train 1.5538296988606453, test 1.5202088356018066, accuracy on test 45.01
2
loss: train 1.4273329628705977, test 1.4282317161560059, accuracy on test 47.71
3
loss: train 1.351450047016144, test 1.356316328048706, accuracy on test 50.33
4
loss: train 1.2709392444193364, test 1.2559773921966553, accuracy on test 54.7
5
loss: train 1.189545194208622, test 1.2133811712265015, accuracy on test 56.01
6
loss: train 1.1402035234868526, test 1.1764880418777466, accuracy on test 58.26
7
loss: train 1.1015233540982008, test 1.131168007850647, accuracy on test 59.28
8
loss: train 1.0488100021481515, test 1.1175953149795532, accuracy on test 60.58
9
loss: train 1.007119820728898, test 1.0910253524780273, accuracy on test 61.79
10
loss: train 0.9895301367491484, test 1.0946228504180908, accuracy on test 61.7
11
loss: train 0.936082072883

(61.94, 1.0793827772140503)

In [ ]:
reporter = JupyterNotebookReporter(max_progress_rows=20,overwrite=True,max_report_frequency  = 10,max_error_rows=5)
scheduler = HyperBandScheduler (
    metric="loss",
        mode="min",
        max_t=2)
config = {"lr": tune.loguniform(1e-4, 1e-1), "momentum":tune.uniform(0.6,0.9),'batch_size':tune.grid_search([2,4,6,8])}
#config = {"lr": tune.grid_search([0.002]), "momentum":tune.grid_search([0.7]),'batch_size':tune.grid_search([10])}
#print(f'Best loss {train_with_parameters(config,train_loader, validation_loader,3)}')
analysis = tune.run(
    tune.with_parameters(train_with_parameters),
     scheduler=scheduler,
     num_samples=10,
    config=config,
    progress_reporter=reporter
   )


2022-03-13 11:14:19,076	INFO hyperband.py:177 -- Bracket too small - Retrying...
2022-03-13 11:14:19,085	INFO hyperband.py:177 -- Bracket too small - Retrying...
2022-03-13 11:14:19,092	INFO hyperband.py:177 -- Bracket too small - Retrying...
2022-03-13 11:14:19,103	INFO hyperband.py:177 -- Bracket too small - Retrying...
2022-03-13 11:14:19,112	INFO hyperband.py:177 -- Bracket too small - Retrying...
2022-03-13 11:14:19,120	INFO hyperband.py:177 -- Bracket too small - Retrying...
2022-03-13 11:14:19,124	INFO hyperband.py:177 -- Bracket too small - Retrying...


In [ ]:
import matplotlib
dfs = analysis.trial_dataframes
ax = None  # This plots everything on the same plot
for d in dfs.values():
    print(d.mean_accuracy)